# Check Pointing Models

In [1]:
mkdir -p weights/improvements

In [3]:
ls weights/

improvements/


### 1 Checkpointing Neural Network Model Improvements

In [4]:
# from conv import MiniVGGNet
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import os

Load CIFAR-10 dataset

In [5]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()

170500096/170498071 [==============================] - 5s 0us/step


In [6]:
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [7]:
trainY = to_categorical(trainY)
testY = to_categorical(testY)

Define model

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

class MiniVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # first CONV => RELU => CONV => RELU => POOL layer set
        model.add(Conv2D(32, (3, 3), padding="same",
            input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # second CONV => RELU => CONV => RELU => POOL layer set
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model

Compile model

In [9]:
opt = SGD(lr=0.01, decay=0.01/40, momentum=0.9, nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", 
              optimizer=opt,
              metrics=["accuracy"])

In [12]:
# insert code
fname = os.path.sep.join(['weights/improvements', 
                  'weights-{epoch:03d}-{val_loss:.4f}.hdf5'])

In [13]:
checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min",
                             save_best_only=True, verbose=1)
callbacks = [checkpoint]

Training network

In [16]:
H = model.fit(trainX, trainY, validation_data=(testX, testY),
    batch_size=64, epochs=40, callbacks=callbacks, verbose=1)

Epoch 1/40
780/782 [============================>.] - ETA: 0s - loss: 0.5534 - accuracy: 0.8027
Epoch 00001: val_loss did not improve from 0.65832
782/782 [==============================] - 6s 7ms/step - loss: 0.5533 - accuracy: 0.8027 - val_loss: 0.6595 - val_accuracy: 0.7694
Epoch 2/40
781/782 [============================>.] - ETA: 0s - loss: 0.5343 - accuracy: 0.8104
Epoch 00002: val_loss improved from 0.65832 to 0.64713, saving model to weights/improvements/weights-002-0.6471.hdf5
782/782 [==============================] - 6s 7ms/step - loss: 0.5343 - accuracy: 0.8104 - val_loss: 0.6471 - val_accuracy: 0.7789
Epoch 3/40
779/782 [============================>.] - ETA: 0s - loss: 0.5124 - accuracy: 0.8194
Epoch 00003: val_loss improved from 0.64713 to 0.64339, saving model to weights/improvements/weights-003-0.6434.hdf5
782/782 [==============================] - 6s 7ms/step - loss: 0.5125 - accuracy: 0.8194 - val_loss: 0.6434 - val_accuracy: 0.7794
Epoch 4/40
779/782 [==============

KeyboardInterrupt: ignored

In [18]:
ls -lh weights/improvements/

total 283M
-rw-r--r-- 1 root root 17M Aug 21 00:52 weights-001-1.4745.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:53 weights-002-0.6471.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:52 weights-002-1.3028.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:53 weights-003-0.6434.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:52 weights-003-1.0667.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:53 weights-004-0.6234.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:52 weights-005-0.8974.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:52 weights-006-0.8662.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:54 weights-007-0.6151.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:52 weights-007-0.8304.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:54 weights-008-0.6074.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:52 weights-008-0.7753.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:52 weights-009-0.7441.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:52 weights-010-0.7153.hdf5
-rw-r--r-- 1 root root 17M Aug 21 00:52 weights-011-0.6875.hdf5
-rw-r--r-- 1 root root 17M Au

check exercise_4_checkpoint/weights/improvements directory
> ls -lh exercise_4_checkpoint/weights/improvements

### 2 Checkpointing Best Neural Network Only

In [ ]:
mkdir -p weights/best

Compiling model

In [ ]:
opt = SGD(lr=0.01, decay=0.01/40, momentum=0.9, nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", 
              optimizer=opt,
              metrics=["accuracy"])

In [ ]:
checkpoint = ModelCheckpoint("weights/best/cifar10_best_weights.hdf5", 
                             monitor="val_loss", mode="min",
                             save_best_only=True, verbose=0)
callbacks = [checkpoint]

Training network

In [ ]:
H = model.fit(trainX, trainY, validation_data=(testX, testY),
                batch_size=64, epochs=40, callbacks=callbacks, verbose=1)